# Prerocessing Data


In [3]:
###############################################################################
# data_preprocessing.py
#
# Steps:
#   1) Load 5 CSV datasets: KSI, TMC, Toronto Police Collisions,
#      Env Canada Weather, ERA5 Weather
#   2) Inspect random samples
#   3) Filter all data to 2015–2020
#   4) Subset columns to only those needed for traffic/weather modeling
#   5) Normalize coordinates:
#       - KSI geometry -> (lon, lat)
#       - TMC rename "longitude"->"lon", "latitude"->"lat"
#       - Collisions: keep them as is (0,0 if missing, no geocoding)
#       - Env Canada rename "x"->"lon", "y"->"lat"
#       - ERA5 parse .geo -> (lon, lat)
#   6) Round timestamps to hourly
#   7) [Optional] Aggregate TMC from 15-min to hourly
#   8) [Optional] Basic Weather transformations (ERA5 Kelvin->Celsius, etc.)
#   9) Print final shapes, heads
###############################################################################

import pandas as pd
import numpy as np
import ast
from IPython.display import display, Markdown
from datetime import datetime, timedelta

###############################################################################
# 1) LOAD DATA
###############################################################################

def load_datasets():
    print("Loading datasets...")
    df_ksi = pd.read_csv("Motor Vehicle Collisions with KSI Data - 4326.csv")
    df_tmc = pd.read_csv("tmc_raw_data_2010_2019.csv")
    df_collisions = pd.read_csv("Traffic_Collisions_Toronto_data.csv")
    df_env = pd.read_csv("hourly_final.csv")  # Env Canada Weather
    df_era5 = pd.read_csv("ERA.csv")          # ERA5 from GEE
    return df_ksi, df_tmc, df_collisions, df_env, df_era5


###############################################################################
# 2) INSPECT SAMPLES
###############################################################################

def show_random_unique_rows(df, title, n=10, subset=None):
    """
    Displays n unique random rows from df for quick inspection.
    If subset is provided, ensures uniqueness first by those columns.
    """
    if subset:
        df = df.drop_duplicates(subset=subset)
    else:
        df = df.drop_duplicates()
    sample = df.sample(min(n, len(df)), random_state=42)
    display(Markdown(f"### {title} – {len(sample)} Unique Rows"))
    display(sample.reset_index(drop=True))

def inspect_data(df_ksi, df_tmc, df_collisions, df_env, df_era5):
    show_random_unique_rows(df_ksi, "KSI Dataset", n=10, subset=["STREET1","STREET2"])
    show_random_unique_rows(df_tmc, "TMC Dataset", n=10, subset=["location_name"])
    show_random_unique_rows(df_collisions, "Toronto Police Collision Dataset", n=10, subset=["Neighbourhood"])
    show_random_unique_rows(df_env, "Env Canada Weather Dataset", n=10)
    show_random_unique_rows(df_era5, "ERA5 GEE Weather Dataset", n=10)


###############################################################################
# 3) DATE/TIME FILTER
###############################################################################

def to_datetime_and_filter(df, date_col, start="2015-01-01", end="2020-12-31"):
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df[date_col] = df[date_col].dt.tz_localize(None)  # remove tz if present
    mask = (df[date_col] >= pd.Timestamp(start)) & (df[date_col] <= pd.Timestamp(end))
    return df[mask].copy()


###############################################################################
# 4) SUBSET COLUMNS
###############################################################################

def subset_columns(df_ksi, df_tmc, df_collisions, df_env, df_era5):
    """
    Keep only the columns directly relevant to:
      - KSI (collision severity, environment factors)
      - TMC (traffic volumes)
      - Collisions (time, location, severity)
      - Env Canada (key weather variables)
      - ERA5 (key weather variables)
    Adjust as needed for your final model goals.
    """
    # --- KSI ---
    ksi_keep = [
        'DATE', 'TIME', 'geometry',  # For time + coords
        'ACCLASS', 'IMPACTYPE', 'VEHTYPE', 'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE',
        'LIGHT', 'RDSFCOND', 'VISIBILITY', 'INJURY', 'FATAL_NO'
    ]
    df_ksi = df_ksi[[c for c in ksi_keep if c in df_ksi.columns]].copy()

    # --- TMC ---
    tmc_keep = ['count_date', 'start_time', 'end_time', 'longitude', 'latitude']
    tmc_volume_cols = [col for col in df_tmc.columns 
                       if any(prefix in col for prefix in ['n_appr_','s_appr_','e_appr_','w_appr_'])]
    df_tmc = df_tmc[tmc_keep + tmc_volume_cols].copy()

    # --- Collisions ---
    collisions_keep = [
        'OccurrenceDate', 'Neighbourhood',  # we won't geocode, just keep if needed for analysis
        'Longitude', 'Latitude',
        'Fatalities', 'Injury_Collisions', 'PD_Collisions'
    ]
    df_collisions = df_collisions[[c for c in collisions_keep if c in df_collisions.columns]].copy()

    # --- Env Canada ---
    env_keep = [
        'LOCAL_DATE', 'TEMP', 'WINDCHILL', 'PRECIP_AMOUNT',
        'RELATIVE_HUMIDITY', 'VISIBILITY', 'WEATHER_ENG_DESC',
        'x','y'
    ]
    df_env = df_env[[c for c in env_keep if c in df_env.columns]].copy()

    # --- ERA5 ---
    era5_keep = [
        'timestamp', 'temperature_2m', 'dewpoint_temperature_2m',
        'total_precipitation', 'u_component_of_wind_10m',
        'v_component_of_wind_10m', '.geo'
    ]
    df_era5 = df_era5[[c for c in era5_keep if c in df_era5.columns]].copy()

    return df_ksi, df_tmc, df_collisions, df_env, df_era5


###############################################################################
# 5) COORDINATE NORMALIZATION
###############################################################################

def parse_ksi_geometry(geom_str):
    """ Parse KSI geometry -> (lon, lat). """
    if pd.isna(geom_str):
        return pd.Series([None, None])
    try:
        geom_dict = ast.literal_eval(geom_str)
        coords = geom_dict.get("coordinates", None)
        if isinstance(coords, list) and len(coords) > 0:
            first_elem = coords[0]
            if (isinstance(first_elem, list) and len(first_elem) > 0 and
                isinstance(first_elem[0], list)):
                # Possibly a MultiLineString
                first_pair = first_elem[0]
            else:
                first_pair = first_elem
            if isinstance(first_pair, list) and len(first_pair) == 2:
                return pd.Series([float(first_pair[0]), float(first_pair[1])])
    except:
        pass
    return pd.Series([None, None])

def normalize_coordinates(df_ksi, df_tmc, df_collisions, df_env, df_era5):
    """
    - KSI: parse geometry -> (lon, lat)
    - TMC: rename (longitude->lon, latitude->lat)
    - Collisions: keep as is, rename (Longitude->lon, Latitude->lat) if present
    - Env Canada: rename (x->lon, y->lat) if present
    - ERA5: parse .geo -> (lon, lat)
    """

    # === KSI: parse geometry ===
    if 'geometry' in df_ksi.columns:
        df_ksi[['lon','lat']] = df_ksi['geometry'].apply(parse_ksi_geometry)
        df_ksi.drop(columns=['geometry'], inplace=True, errors='ignore')

    # === TMC: rename ===
    if 'longitude' in df_tmc.columns:
        df_tmc.rename(columns={'longitude':'lon','latitude':'lat'}, inplace=True)

    # === Collisions: just rename "Longitude","Latitude" => "lon","lat" if present
    if 'Longitude' in df_collisions.columns:
        df_collisions.rename(columns={'Longitude':'lon'}, inplace=True)
    if 'Latitude' in df_collisions.columns:
        df_collisions.rename(columns={'Latitude':'lat'}, inplace=True)
    # (We do NOT attempt geocoding for 0,0 or NaN in collisions here.)

    # === Env Canada: rename x->lon, y->lat if present
    if 'x' in df_env.columns and 'y' in df_env.columns:
        df_env.rename(columns={'x':'lon','y':'lat'}, inplace=True)

    # === ERA5: parse .geo -> (lon, lat)
    if '.geo' in df_era5.columns:
        def parse_era5_geo(geo_str):
            if pd.isna(geo_str):
                return pd.Series([None, None])
            try:
                geo_dict = ast.literal_eval(geo_str)
                coords = geo_dict.get("coordinates", None)
                if coords and len(coords) == 2:
                    return pd.Series([float(coords[0]), float(coords[1])])
            except:
                pass
            return pd.Series([None, None])

        df_era5[['lon','lat']] = df_era5['.geo'].apply(parse_era5_geo)
        df_era5.drop(columns=['.geo'], inplace=True)

    return df_ksi, df_tmc, df_collisions, df_env, df_era5


###############################################################################
# 6) TIME NORMALIZATION (ROUND TO HOUR)
###############################################################################

def round_all_times(df_ksi, df_tmc, df_collisions, df_env, df_era5):
    """
    Round all timestamps to hourly:
      - KSI: combine DATE + TIME -> 'datetime'
      - TMC: parse 'start_time'
      - Collisions: parse 'OccurrenceDate'
      - Env Canada: parse 'LOCAL_DATE'
      - ERA5: parse 'timestamp'
    """

    # --- KSI ---
    if 'DATE' in df_ksi.columns and 'TIME' in df_ksi.columns:
        def combine_ksi_time(row):
            if pd.isna(row['DATE']):
                return pd.NaT
            t_str = str(row['TIME']).zfill(4)  # e.g. 624 -> '0624'
            hh = int(t_str[:-2]) if len(t_str) >= 2 else 0
            mm = int(t_str[-2:]) if len(t_str) >= 2 else 0
            dt = row['DATE'] + pd.Timedelta(hours=hh, minutes=mm)
            return dt

        df_ksi['datetime'] = df_ksi.apply(combine_ksi_time, axis=1)
        df_ksi['datetime'] = pd.to_datetime(df_ksi['datetime'], errors='coerce')
        df_ksi['datetime'] = df_ksi['datetime'].dt.round('H')

    # --- TMC ---
    if 'start_time' in df_tmc.columns:
        df_tmc['start_time'] = pd.to_datetime(df_tmc['start_time'], errors='coerce')
        df_tmc['start_time'] = df_tmc['start_time'].dt.round('H')

    # --- Collisions ---
    if 'OccurrenceDate' in df_collisions.columns:
        df_collisions['OccurrenceDate'] = pd.to_datetime(df_collisions['OccurrenceDate'], errors='coerce')
        df_collisions['OccurrenceDate'] = df_collisions['OccurrenceDate'].dt.round('H')

    # --- Env Canada ---
    if 'LOCAL_DATE' in df_env.columns:
        df_env['LOCAL_DATE'] = pd.to_datetime(df_env['LOCAL_DATE'], errors='coerce')
        df_env['LOCAL_DATE'] = df_env['LOCAL_DATE'].dt.round('H')

    # --- ERA5 ---
    if 'timestamp' in df_era5.columns:
        df_era5['timestamp'] = pd.to_datetime(df_era5['timestamp'], errors='coerce')
        df_era5['timestamp'] = df_era5['timestamp'].dt.round('H')

    return df_ksi, df_tmc, df_collisions, df_env, df_era5


###############################################################################
# 7) (Optional) AGGREGATE TMC TO HOURLY
###############################################################################

def aggregate_tmc_hourly(df_tmc):
    """
    Converts TMC from 15-min intervals to hourly by summing volumes across
    each hour. This is optional but often useful if your weather & collisions
    are hourly.
    - Summarizes direction-based columns (cars, trucks, etc.).
    - Group by (lon, lat, 'hour_start').

    Returns df_tmc_hourly with columns:
        ['lon', 'lat', 'hour_start', <summed volume columns>].
    """
    if 'start_time' not in df_tmc.columns:
        print("Warning: TMC has no 'start_time' column, skipping aggregation.")
        return df_tmc

    # Create a new column 'hour_start' from 'start_time'
    df_tmc['hour_start'] = df_tmc['start_time']

    # Identify columns that hold numeric traffic volumes
    vol_cols = [
        col for col in df_tmc.columns 
        if any(prefix in col for prefix in ['n_appr_','s_appr_','e_appr_','w_appr_'])
    ]

    group_cols = ['lon','lat','hour_start']
    df_agg = df_tmc.groupby(group_cols, dropna=False)[vol_cols].sum().reset_index()

    # Example: rename group_cols if you'd like
    # df_agg.rename(columns={'hour_start':'datetime'}, inplace=True)

    return df_agg


###############################################################################
# 8) (Optional) Weather Transformations
###############################################################################

def transform_weather_era5(df_era5):
    """
    Example: Convert ERA5 temperature_2m from Kelvin to Celsius, etc.
    Create new columns like 'temp_c' and 'wind_speed'.
    """
    if 'temperature_2m' in df_era5.columns:
        df_era5['temp_c'] = df_era5['temperature_2m'] - 273.15

    if 'u_component_of_wind_10m' in df_era5.columns and 'v_component_of_wind_10m' in df_era5.columns:
        df_era5['wind_speed_10m'] = np.sqrt(
            df_era5['u_component_of_wind_10m']**2 + 
            df_era5['v_component_of_wind_10m']**2
        )

    return df_era5


###############################################################################
# MAIN
###############################################################################

def main():
    # ----------- 1) LOAD -----------
    df_ksi, df_tmc, df_collisions, df_env, df_era5 = load_datasets()
    print('Size of datasets: ', df_ksi.shape, df_tmc.shape, df_collisions.shape, df_env.shape, df_era5.shape)
    print('KSI columns: ', df_ksi.columns)
    print('TMC columns: ', df_tmc.columns)  
    print('Collisions columns: ', df_collisions.columns)
    print('Env Canada columns: ', df_env.columns)
    print('ERA5 columns: ', df_era5.columns)
    
    # ----------- 2) INSPECT -----------
    inspect_data(df_ksi, df_tmc, df_collisions, df_env, df_era5)

    # ----------- 3) FILTER 2015-2020 -----------
    df_ksi = to_datetime_and_filter(df_ksi, 'DATE')
    df_tmc = to_datetime_and_filter(df_tmc, 'count_date')
    df_collisions = to_datetime_and_filter(df_collisions, 'OccurrenceDate')
    df_env = to_datetime_and_filter(df_env, 'LOCAL_DATE')
    df_era5 = to_datetime_and_filter(df_era5, 'timestamp')

    # ----------- 4) SUBSET COLUMNS -----------
    df_ksi, df_tmc, df_collisions, df_env, df_era5 = subset_columns(
        df_ksi, df_tmc, df_collisions, df_env, df_era5
    )

    # ----------- 5) COORD NORMALIZATION -----------
    df_ksi, df_tmc, df_collisions, df_env, df_era5 = normalize_coordinates(
        df_ksi, df_tmc, df_collisions, df_env, df_era5
    )

    # ----------- 6) TIME NORMALIZATION -----------
    df_ksi, df_tmc, df_collisions, df_env, df_era5 = round_all_times(
        df_ksi, df_tmc, df_collisions, df_env, df_era5
    )

    # ----------- 7) (Optional) AGGREGATE TMC TO HOURLY -----------
    # If you want to combine TMC with hourly collisions/weather:
    df_tmc_hourly = aggregate_tmc_hourly(df_tmc)

    # ----------- 8) (Optional) WEATHER TRANSFORMATIONS -----------
    df_era5 = transform_weather_era5(df_era5)
    # Similarly, you could transform Env Canada if needed
    # e.g. parse TEMP flags or unify units with ERA5

    # ----------- PRINT RESULTS -----------
    print("\n===== FINAL DATAFRAMES =====")
    print(f"KSI shape: {df_ksi.shape}")
    print(f"TMC shape (raw 15-min or partial): {df_tmc.shape}")
    print(f"TMC hourly shape (aggregated): {df_tmc_hourly.shape}")
    print(f"Collisions shape: {df_collisions.shape}")
    print(f"Env Canada shape: {df_env.shape}")
    print(f"ERA5 shape: {df_era5.shape}")

    print("\n--- KSI HEAD ---")
    print(df_ksi.head())

    print("\n--- TMC HEAD (15-min intervals) ---")
    print(df_tmc.head())

    print("\n--- TMC HOURLY HEAD (aggregated) ---")
    print(df_tmc_hourly.head())

    print("\n--- Collisions HEAD (no geocoding) ---")
    print(df_collisions.head())

    print("\n--- Env Canada HEAD ---")
    print(df_env.head())

    print("\n--- ERA5 HEAD (with optional transforms) ---")
    print(df_era5.head())


if __name__ == "__main__":
    main()


Loading datasets...


/var/folders/r0/10rng5012t73v6tl4z4_sqp00000gn/T/ipykernel_5809/2362808785.py:37: DtypeWarning: Columns (4,8,17,19,20,24,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df_env = pd.read_csv("hourly_final.csv")  # Env Canada Weather


Size of datasets:  (18957, 50) (223817, 55) (499538, 19) (97782, 37) (526080, 10)
KSI columns:  Index(['_id', 'ACCNUM', 'DATE', 'TIME', 'STREET1', 'STREET2', 'OFFSET',
       'ROAD_CLASS', 'DISTRICT', 'ACCLOC', 'TRAFFCTL', 'VISIBILITY', 'LIGHT',
       'RDSFCOND', 'ACCLASS', 'IMPACTYPE', 'INVTYPE', 'INVAGE', 'INJURY',
       'FATAL_NO', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND',
       'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT', 'CYCCOND',
       'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK',
       'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER', 'SPEEDING', 'AG_DRIV',
       'REDLIGHT', 'ALCOHOL', 'DISABILITY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'DIVISION', 'geometry'],
      dtype='object')
TMC columns:  Index(['_id', 'count_id', 'count_date', 'location_name', 'longitude',
       'latitude', 'centreline_type', 'centreline_id', 'px', 'start_time',
       'end_time', 'n_appr_cars_r', 'n_appr_cars_t', 'n_appr_car

### KSI Dataset – 10 Unique Rows

,_id,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,ACCLOC,...,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,geometry
0,4447,1.097712e+06,2009-04-04,228,LIPPINCOTT ST,BLOOR ST W,NaN,Major Arterial,Toronto and East York,NaN,...,NaN,NaN,Yes,NaN,79,University,79,University (79),D14,"{""coordinates"": [[-79.4099900003758, 43.665344..."
1,259,8.991260e+05,2006-03-26,1614,CALEDONIA RD,NORMAN AVE,NaN,Minor Arterial,Toronto and East York,NaN,...,NaN,NaN,NaN,NaN,92,Corso Italia-Davenport,92,Corso Italia-Davenport (92),D13,"{""coordinates"": [[-79.4557899995631, 43.677744..."
2,11502,NaN,2015-04-22,1402,3 RAINIER SQ,NaN,4 m South of,Local,Scarborough,Private Driveway,...,Yes,NaN,NaN,NaN,148,East L'Amoreaux,117,L'Amoreaux (117),D42,"{""coordinates"": [[-79.3030119995988, 43.794242..."
3,15991,NaN,2019-10-05,1611,DUPONT ST,BEDFORD RD,NaN,Minor Arterial,Toronto and East York,At Intersection,...,NaN,NaN,NaN,NaN,95,Annex,95,Annex (95),D53,"{""coordinates"": [[-79.4005079994135, 43.676299..."
4,15656,NaN,2019-06-20,1315,REGENT PARK BLVD,NaN,NaN,Major Arterial,Toronto and East York,At Intersection,...,NaN,NaN,NaN,NaN,72,Regent Park,72,Regent Park (72),D51,"{""coordinates"": [[-79.3617430003326, 43.660287..."
5,16973,1.000499e+09,2021-03-18,1056,GREENWIN VILLAGE RD,BATHURST ST,50 m East of,Collector,North York,At/Near Private Drive,...,Yes,NaN,NaN,NaN,36,Newtonbrook West,36,Newtonbrook West (36),D32,"{""coordinates"": [[-79.4448350001569, 43.790754..."
6,6095,1.180034e+06,2010-07-16,1011,SPADINA AVE,ST ANDREW ST,NaN,Major Arterial,Toronto and East York,NaN,...,NaN,NaN,NaN,NaN,78,Kensington-Chinatown,78,Kensington-Chinatown (78),D52,"{""coordinates"": [[-79.3985930000871, 43.654345..."
7,8566,1.315841e+06,2012-08-12,951,CHURCH ST,CARLTON ST,NaN,Major Arterial,Toronto and East York,At Intersection,...,Yes,Yes,NaN,NaN,168,Downtown Yonge East,75,Church-Yonge Corridor (75),D51,"{""coordinates"": [[-79.3793900004206, 43.661845..."
8,2426,9.904020e+05,2007-08-27,1441,FINCH Aven E,LESLIE Stre,NaN,Major Arterial,North York,At Intersection,...,NaN,NaN,NaN,NaN,49,Bayview Woods-Steeles,49,Bayview Woods-Steeles (49),D33,"{""coordinates"": [[-79.3680900003581, 43.790045..."
9,7158,1.251049e+06,2011-07-17,630,BREMNER Boul,VAN DE WATER Cres,NaN,Minor Arterial,Toronto and East York,At Intersection,...,Yes,NaN,NaN,NaN,165,Harbourfront-CityPlace,77,Waterfront Communities-The Island (77),D52,"{""coordinates"": [[-79.3889900000155, 43.640345..."


...

,x,y,LOCAL_DATE,STATION_PRESSURE,TEMP_FLAG,WINDCHILL,LOCAL_HOUR,RELATIVE_HUMIDITY,WIND_DIRECTION_FLAG,WIND_DIRECTION,...,LOCAL_DAY,PROVINCE_CODE,UTC_DATE,DEW_POINT_TEMP,TEMP,WINDCHILL_FLAG,VISIBILITY,RELATIVE_HUMIDITY_FLAG,HUMIDEX,VISIBILITY_FLAG
0,-79.4,43.666667,2018-11-23 17:00:00,100.79,NaN,NaN,17,60.0,M,NaN,...,23,ON,2018-11-23T22:00:00,-6.0,0.8,NaN,NaN,NaN,NaN,NaN
1,-79.4,43.666667,2024-06-04 15:00:00,99.96,NaN,NaN,15,56.0,NaN,NaN,...,4,ON,2024-06-04T20:00:00,16.9,26.4,NaN,NaN,NaN,32.0,NaN
2,-79.4,43.666667,2020-06-24 16:00:00,99.37,NaN,NaN,16,34.0,NaN,NaN,...,24,ON,2020-06-24T21:00:00,6.2,22.7,NaN,NaN,NaN,NaN,NaN
3,-79.4,43.666667,2018-02-15 13:00:00,99.11,NaN,NaN,13,75.0,M,NaN,...,15,ON,2018-02-15T18:00:00,3.8,7.9,NaN,NaN,NaN,NaN,NaN
4,-79.4,43.666667,2014-05-12 17:00:00,100.44,NaN,NaN,17,37.0,M,NaN,...,12,ON,2014-05-12T22:00:00,2.0,16.7,NaN,NaN,NaN,NaN,NaN
5,-79.4,43.666667,2017-01-14 08:00:00,102.25,NaN,NaN,8,66.0,M,NaN,...,14,ON,2017-01-14T13:00:00,-13.2,-7.9,NaN,NaN,NaN,NaN,NaN
6,-79.4,43.666667,2025-02-15 22:00:00,99.70,NaN,NaN,22,88.0,NaN,NaN,...,15,ON,2025-02-16T03:00:00,-4.4,-2.7,NaN,NaN,NaN,NaN,NaN
7,-79.4,43.666667,2017-03-28 12:00:00,100.16,NaN,NaN,12,69.0,M,NaN,...,28,ON,2017-03-28T17:00:00,2.6,8.0,NaN,NaN,NaN,NaN,NaN
8,-79.4,43.666667,2019-07-28 18:00:00,100.07,NaN,NaN,18,49.0,NaN,NaN,...,28,ON,2019-07-28T23:00:00,16.0,27.5,NaN,NaN,NaN,32.0,NaN
9,-79.4,43.666667,2015-02-08 19:00:00,100.12,NaN,NaN,19,84.0,M,NaN,...,8,ON,2015-02-09T00:00:00,-12.9,-10.6,NaN,NaN,NaN,NaN,NaN


### ERA5 GEE Weather Dataset – 10 Unique Rows

,system:index,dewpoint_temperature_2m,location,surface_pressure,temperature_2m,timestamp,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m,.geo
0,9_20190303T01,268.240677,Agincourt,100170.890625,270.709167,2019-03-03 01:00,2.066698e-05,2.189865,-3.053253,"{""type"":""Point"",""coordinates"":[-79.2939,43.7996]}"
1,4_20200706T14,291.267044,East York,100313.261719,296.455444,2020-07-06 14:00,8.583069e-07,-4.281723,0.109268,"{""type"":""Point"",""coordinates"":[-79.3017,43.665]}"
2,5_20151230T10,273.556503,Guildwood,100968.562500,275.845764,2015-12-30 10:00,2.257153e-04,5.781815,3.189896,"{""type"":""Point"",""coordinates"":[-79.1845,43.7636]}"
3,5_20200510T02,269.013580,Guildwood,100428.437500,276.657867,2020-05-10 02:00,0.000000e+00,6.243011,-1.323898,"{""type"":""Point"",""coordinates"":[-79.1845,43.7636]}"
4,1_20201202T02,269.730377,North York,98513.074219,272.274414,2020-12-02 02:00,5.876273e-05,4.092887,-4.099899,"{""type"":""Point"",""coordinates"":[-79.5181,43.7731]}"
5,5_20151017T00,273.650330,Guildwood,100559.398438,281.555420,2015-10-17 00:00,7.847980e-04,7.381165,-4.987549,"{""type"":""Point"",""coordinates"":[-79.1845,43.7636]}"
6,6_20150111T13,261.082565,High Park,101304.058594,265.088928,2015-01-11 13:00,4.999340e-06,3.057251,3.387320,"{""type"":""Point"",""coordinates"":[-79.4309,43.6816]}"
7,3_20170814T08,288.141953,Etobicoke,99507.597656,291.820892,2017-08-14 08:00,0.000000e+00,0.243805,-0.874802,"{""type"":""Point"",""coordinates"":[-79.62050000000..."
8,4_20191107T13,270.301544,East York,100867.046875,273.623306,2019-11-07 13:00,6.001830e-03,-0.419662,-4.553178,"{""type"":""Point"",""coordinates"":[-79.3017,43.665]}"
9,4_20190805T01,289.511612,East York,100103.890625,295.110657,2019-08-05 01:00,0.000000e+00,0.110672,0.722921,"{""type"":""Point"",""coordinates"":[-79.3017,43.665]}"



===== FINAL DATAFRAMES =====
KSI shape: (5571, 16)
TMC shape (raw 15-min or partial): (122570, 50)
TMC hourly shape (aggregated): (42167, 47)
Collisions shape: (417795, 7)
Env Canada shape: (52355, 9)
ERA5 shape: (525850, 10)

--- KSI HEAD ---
            DATE  TIME           ACCLASS         IMPACTYPE  \
11303 2015-01-01   624  Non-Fatal Injury       Approaching   
11304 2015-01-01   624  Non-Fatal Injury       Approaching   
11305 2015-01-01   624  Non-Fatal Injury       Approaching   
11306 2015-01-02   949  Non-Fatal Injury  Turning Movement   
11307 2015-01-02   949  Non-Fatal Injury  Turning Movement   

                            VEHTYPE PEDESTRIAN CYCLIST AUTOMOBILE  \
11303     Automobile, Station Wagon        NaN     NaN        Yes   
11304     Automobile, Station Wagon        NaN     NaN        Yes   
11305                           NaN        NaN     NaN        Yes   
11306     Automobile, Station Wagon        NaN     NaN        Yes   
11307  Truck - Closed (Blazer, etc)  

/var/folders/r0/10rng5012t73v6tl4z4_sqp00000gn/T/ipykernel_5809/2362808785.py:234: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_ksi['datetime'] = df_ksi['datetime'].dt.round('H')
/var/folders/r0/10rng5012t73v6tl4z4_sqp00000gn/T/ipykernel_5809/2362808785.py:239: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_tmc['start_time'] = df_tmc['start_time'].dt.round('H')
/var/folders/r0/10rng5012t73v6tl4z4_sqp00000gn/T/ipykernel_5809/2362808785.py:244: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_collisions['OccurrenceDate'] = df_collisions['OccurrenceDate'].dt.round('H')
/var/folders/r0/10rng5012t73v6tl4z4_sqp00000gn/T/ipykernel_5809/2362808785.py:249: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_env['LOCAL_DATE'] = df_env['LOCAL_DATE'].dt.round('H')
/var/folders/r0/10